In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.subplots as subplt
import plotly.express as px
import plotly.graph_objs as go
import missingno as msno
import plotly.offline as pyo
import klib
%matplotlib inline

pyo.init_notebook_mode()

In [ ]:
import sys
print (sys.version)

In [ ]:
salaries = pd.read_csv("https://raw.githubusercontent.com/samburu/photo/master/IT%20Salary%20Survey%20EU%20%202020.csv")

In [ ]:
msno.bar(salaries)

# Data Exploration

In [ ]:
salaries.head(5)

In [ ]:
salaries.shape

In [ ]:
salaries.columns.to_list()

In [ ]:
# Rename columns in order to make display on graphs
# more user friendly
salaries.rename(
    columns={
        "Total Years of Experience": "Experience(Years)",
        "Your main technology / programming language": "PL",
        "Other technologies/programming languages you use often": "PL (Other)",
        "Yearly brutto salary (without bonus and stocks) in EUR": "Salary (2020)",
        "Yearly bonus + stocks in EUR": "Bonus (2020)",
        "Annual brutto salary (without bonus and stocks) one year ago. Only answer if staying in the same country": "Salary (2019)",
        "Number of vacation days": "Vacation (days)",
        "Annual bonus+stocks one year ago. Only answer if staying in same country": "Bonus+Stocks (2019)",
        "Main language at work": "PL (Work)",
        "Position ": "Position",
        "Have you lost your job due to the coronavirus outbreak?": "Job Loss? (COVID)",
        "Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week": "Shorter Work Week (hrs/week)",
        "Have you received additional monetary support from your employer due to Work From Home? If yes, how much in 2020 in EUR": "Extra perks (2020)",
    }, inplace=True)

In [ ]:
salaries.columns.to_list()

In [ ]:
# Missing values as %
salaries.isna().mean().round(4) * 100

In [ ]:
# Visualizing the missing values
msno.bar(salaries)

In [ ]:
klib.missingval_plot(salaries)

In [ ]:
px.box(salaries, y='Age', orientation='v',
                    color="Gender")

Distribution of Age

In [ ]:
# Showing the distribution
# The hue - colors - indicate gender
fig = subplt.make_subplots(rows=4, shared_xaxes=True)

for i, data in enumerate(px.box(salaries, x='Age', orientation='h',
                    color="Gender").data):
    fig.add_trace(data, row=i+1, col=1)

for data in px.histogram(salaries, x="Age", color="Gender").data:
    fig.add_trace(data, row=4, col=1)
fig.update_layout(title_text="Distribution By Sex")
fig.show()

In [ ]:
# Different way of showing the above
px.histogram(salaries, x="Age", color="Gender", marginal="violin")

In [ ]:
# Showing the distribution:
klib.dist_plot(salaries)

In [ ]:
# Outliers?
px.box(salaries, x='Salary (2020)')

In [ ]:
salaries.describe()

In [ ]:
def fix_outliers(df, feature, q1, q3, n=1.5):
    df = df.copy()
    p25 = df[feature].quantile(q1)
    p75 = df[feature].quantile(q3)
    iqr = p75 - p25
    upper_limit = p75 + n * iqr
    lower_limit = p25 - n * iqr
    # New DataFrame after removing the outlier 
    new_df = df[(df[feature] > lower_limit) & (df[feature] < upper_limit)]
    # Checking the difference
    diff = df.shape[0] - new_df.shape[0]
    return (diff, new_df)

In [ ]:
rows, salaries = fix_outliers(salaries, 'Salary (2020)', 0.25, 0.75)

In [ ]:
rows

In [ ]:
px.box(salaries, x='Salary (2020)')

In [ ]:
# After dropping salary outliers
klib.dist_plot(salaries)

In [ ]:
# Different way of showing the above
px.histogram(salaries, x="Age", color="Gender", marginal="box")

In [ ]:
salaries.dropna(axis=0, subset=['Seniority level',], inplace=True)

In [ ]:
px.pie(salaries, names='Seniority level')

In [ ]:
salaries['Seniority level'].value_counts()
# salaries['Seniority level']

In [ ]:
salaries['Seniority level'].unique()

In [ ]:
salaries.loc[salaries['Seniority level']== 'Entry level']

In [ ]:
# Rename seniority
_remap = {}
for col in salaries['Seniority level'].unique():
    match col:
        case 'Head':
            _remap[col] = 'Lead'
        case col if any([col in ['VP', 'No level', 'Director', 'Key'],
                        "manager" in str(col).lower()]):
            _remap[col] = 'Manager'
        case col if any([col in ['Principal', 'No level '],
                        "no idea" in str(col).lower()]):
            _remap[col] = 'Senior'
        case col if col in ['Entry level', 'Intern']:
            _remap[col] = 'Junior'
salaries['Seniority level'].replace(_remap.keys(), _remap.values(), inplace=True)

In [ ]:
salaries['Seniority level'].value_counts()


In [ ]:
px.pie(salaries, names='Seniority level')

In [ ]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

salaries.loc[salaries['Seniority level'] == 'No level ']

In [ ]:
pd.options.display.max_rows = 999
salaries['Position'].value_counts()

In [ ]:
_remap = {}
for col in salaries['Position'].unique():
    match col:
        case col if "lead" in str(col).lower():
            _remap[col] = "Team Lead"
        case col if "ios" in str(col).lower():
            _remap[col] = "Mobile Developer"
        case col if col in ['QA Engineer'] or 'test' in (c := str(col).lower()) or 'qa' in c:
            _remap[col] = "QA Engineer"
        case col if col in ["DatabEngineer", "data engineer",
                            "Big Data Engineer", "Senior Data Engineer"]:
            _remap[col] = "Data Engineer"
        case col if any([col in ["Stuttgart", "Recruiter", "Consultant", "Presales Engineer", "Researcher",
                                "Localization producer", "Reporting Engineer", "agile master", "Banker",
                                "Agile Coach", "Scrum Master", "Beikoch", "It Consulting",
                                 "Computational linguist", "Rentner", "Application Consultant", "Professor"],
                         "sales" in (c := str(col).lower()),
                         "consult" in c,
                         "agile" in c,
                         "student" in c,
                         "recruit" in c]):
            _remap[col] = "Other"
        case col if col in ['Fullstack Developer', "IT Spezialist", 'Embedded Software Engineer',
                            'Sofware/Hardware Engineer', 'Software Engineer', "Firmware Engineer", "Hardware Engineer"] or "full" in (c := str(col).lower()) or "java" in c or "data engineer" in c:
            _remap[col] = 'Software/Hardware Engineer'
        case col if any(["head" in (c := str(col).lower()),
                         "manage" in c, "scrum" in c,
                         "cto" in c,
                         "vp" in c, "director" in c,]):
            _remap[col] = 'Manager'
        case col if "insights" in (c := str(col).lower()) or "analyst" in c or "analytics" in c:
            _remap[col] = "Data Analyst"
        case col if col in ['DevOps', 'SRE', 'DBA',
                            "Support Engineer",
                            "support engineer"] \
                  or "security" in (c := str(col).lower()) \
                  or "roboti" in c \
                  or "sap" in c \
                  or "network" in c \
                  or "system" in c \
                  or "cloud" in c:
            _remap[col] = "Infra"
        case col if "archite" in str(col).lower():
            _remap[col] = "Architect"
        case col if col in ['Designer (UI/UX)', 'Graphic Designer']:
            _remap[col] = 'UI/UX'
salaries['Position'].replace(_remap.keys(), _remap.values(), inplace=True)


In [ ]:
# px.bar(salaries.value_counts('Position'), y=salaries['Position'], color = "Gender")
sns.countplot(y="Position", hue="Gender", data=salaries)

In [ ]:
fig = px.bar(salaries.groupby(['Position']).mean().reset_index(), x='Position', y='Salary (2020)')
fig.update_layout(barmode='stack',  xaxis={'categoryorder':'total descending'})
fig.show()
# TODO set start y axis

In [ ]:
px.histogram(salaries, x="Salary (2020)", color="Gender", marginal="box")

In [ ]:
salaries["PL"].value_counts()
# sns.countplot(y="PL", data=salaries)

In [ ]:
_old = salaries["PL"].dropna().unique().tolist()
_new = [val.strip().lower() for val in _old]
salaries['PL'].replace(_old, _new, inplace=True)

In [ ]:
# Main PL for everyone
from wordcloud import WordCloud
plt.figure(num=None,figsize=(100,100),dpi=80,facecolor='w',edgecolor='k')
plt.imshow(WordCloud().generate(' '.join([str(val) for val in salaries['PL'].dropna().values.tolist()])))
plt.axis("off")
plt.show()

In [ ]:
# Backend Devs
plt.figure(num=None,figsize=(100,100),dpi=80,facecolor='w',edgecolor='k')
plt.imshow(WordCloud().generate(' '.join([str(val) for val
                                          in salaries[salaries["Position"] ==
                                                      "Backend Developer"]['PL'].dropna().values.tolist()])))
plt.axis("off")
plt.show()

In [ ]:
# Backend Devs
plt.figure(num=None,figsize=(100,100),dpi=80,facecolor='w',edgecolor='k')
plt.imshow(WordCloud().generate(' '.join([str(val) for val
                                          in salaries[salaries["Position"] ==
                                                      "Frontend Developer"]['PL'].dropna().values.tolist()])))
plt.axis("off")
plt.show()

In [ ]:
# Backend Devs
plt.figure(num=None,figsize=(100,100),dpi=80,facecolor='w',edgecolor='k')
plt.imshow(WordCloud().generate(' '.join([str(val) for val
                                          in salaries[salaries["Position"] ==
                                                      "Infra"]['PL'].dropna().values.tolist()])))
plt.axis("off")
plt.show()

In [ ]:
# Data Scientists
plt.figure(num=None,figsize=(100,100),dpi=80,facecolor='w',edgecolor='k')
plt.imshow(WordCloud().generate(' '.join([str(val) for val
                                          in salaries[salaries["Position"].str.contains("Data|data",
                                                                                        na = False)]['PL'].dropna().values.tolist()])))
plt.axis("off")
plt.show()

In [ ]:
# SW/HW Engineer
plt.figure(num=None,figsize=(100,100),dpi=80,facecolor='w',edgecolor='k')
plt.imshow(WordCloud().generate(' '.join([str(val) for val
                                          in salaries[salaries["Position"].str.contains("Software/Hardware Engineer",
                                                                                        na = False)]['PL'].dropna().values.tolist()])))
plt.axis("off")
plt.show()

In [ ]:
salaries['Total years of experience'].loc[salaries['Total years of experience']=='6 (not as a data scientist, but as a lab scientist)'] = '6'
salaries['Total years of experience'].loc[salaries['Total years of experience']=='less than year'] = '1'
salaries['Total years of experience'].loc[salaries['Total years of experience']=='15, thereof 8 as CTO'] = '15'
salaries['Total years of experience'].loc[salaries['Total years of experience']=='1 (as QA Engineer) / 11 in total'] = '11'
salaries['Total years of experience'].loc[salaries['Total years of experience']=='383'] = None
salaries['Total years of experience'].loc[salaries['Total years of experience']=='1,5'] = '1.5'
salaries['Total years of experience'].loc[salaries['Total years of experience']=='2,5'] = '2.5'


In [ ]:
salaries['Total years of experience'].value_counts()

In [ ]:
salaries["Total years of experience"] = salaries["Total years of experience"].astype("float")

In [ ]:
px.scatter(salaries,
           y="Total years of experience", x="Salary (2020)",
           trendline="ols",
           color="Gender")

In [ ]:
salaries.columns 

In [ ]:
salaries["Company type"].value_counts()